# Microsoft Malware Prediction

#### https://www.kaggle.com/c/microsoft-malware-prediction/

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_rows = 10

In [3]:
df = pd.read_csv("./data/train_un.csv", low_memory=False)#, dtype=dtypes)

In [4]:
df.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000038f24a1ee98931456b3e49f7934b,win8defender,1.1.15200.1,4.18.1806.18062,1.275.879.0,0,7.0,0,NaN,53447.0,...,51032.0,0,0.0,0.0,0,0,0.0,0.0,7.0,0
2,00004a558a836aa7b6ca3a29ed4a0ac3,win8defender,1.1.15100.1,4.12.16299.15,1.273.1622.0,0,7.0,0,NaN,16847.0,...,17595.0,1,0.0,0.0,0,0,0.0,0.0,4.0,1
3,000079a261a28c51c4ae8393c7fd97e8,win8defender,1.1.15000.2,4.10.14393.1593,1.271.613.0,0,7.0,0,NaN,53447.0,...,14728.0,0,0.0,0.0,0,0,0.0,0.0,1.0,0
4,000090c74b057a93e59e7a700c00ea00,win8defender,1.1.13407.0,4.9.10586.589,1.235.1997.0,0,7.0,0,788.0,53447.0,...,12459.0,0,0.0,0.0,1,0,1.0,0.0,1.0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889744 entries, 0 to 889743
Data columns (total 83 columns):
MachineIdentifier                                    889744 non-null object
ProductName                                          889744 non-null object
EngineVersion                                        889744 non-null object
AppVersion                                           889744 non-null object
AvSigVersion                                         889744 non-null object
IsBeta                                               889744 non-null int64
RtpStateBitfield                                     886535 non-null float64
IsSxsPassiveMode                                     889744 non-null int64
DefaultBrowsersIdentifier                            43009 non-null float64
AVProductStatesIdentifier                            886107 non-null float64
AVProductsInstalled                                  886107 non-null float64
AVProductsEnabled                                   

### NaN Count para cada coluna

In [6]:
nan_count = df.isna().sum()
nan_columns = nan_count[lambda x: x > 0]

nan_columns

RtpStateBitfield                             3209
DefaultBrowsersIdentifier                  846735
AVProductStatesIdentifier                    3637
AVProductsInstalled                          3637
AVProductsEnabled                            3637
                                            ...  
Census_IsWIMBootEnabled                    563872
Census_IsVirtualDevice                       1540
Census_IsAlwaysOnAlwaysConnectedCapable      7097
Wdft_IsGamer                                30139
Wdft_RegionIdentifier                       30139
Length: 44, dtype: int64

### Atributos com dados faltantes maiores que 95% serão excluídos

In [7]:
remove_columns = nan_columns[lambda x: x> 880000].keys()

df.drop(columns=remove_columns, inplace=True)

### Atributos com dados faltantes menores que 5% serão substituidos pela  moda

In [8]:
mean_columns = nan_columns[lambda x: x < 45000].keys()
mean_columns

Index(['RtpStateBitfield', 'AVProductStatesIdentifier', 'AVProductsInstalled',
       'AVProductsEnabled', 'CityIdentifier', 'GeoNameIdentifier',
       'OsBuildLab', 'IsProtected', 'IeVerIdentifier', 'Firewall',
       'UacLuaenable', 'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_TotalPhysicalRAM', 'Census_ChassisTypeName',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches',
       'Census_InternalPrimaryDisplayResolutionHorizontal',
       'Census_InternalPrimaryDisplayResolutionVertical',
       'Census_PowerPlatformRoleName', 'Census_InternalBatteryNumberOfCharges',
       'Census_OSInstallLanguageIdentifier', 'Census_IsFlightsDisabled',
       'Census_FirmwareManufacturerIdentifier',
       'Census_FirmwareVersionIdentifier

In [9]:
for column in mean_columns:
    df[column].fillna(df[column].mode()[0], inplace=True)
    print(df[column].mode()[0])

7.0
53447.0
1.0
1.0
130775.0
277.0
17134.1.amd64fre.rs4_release.180410-1804
1.0
137.0
1.0
1.0
2668.0
313586.0
4.0
5.0
2697.0
476940.0
HDD
28542.0
4096.0
Notebook
15.5
1366.0
768.0
Mobile
0.0
8.0
0.0
142.0
33105.0
0.0
0.0
0.0
10.0


### Analise das demais colunas

In [12]:
check_columns = nan_columns.drop(mean_columns)
check_columns = check_columns.drop(remove_columns)
check_columns

DefaultBrowsersIdentifier     846735
OrganizationIdentifier        274148
SMode                          53700
SmartScreen                   316239
Census_InternalBatteryType    631910
Census_IsFlightingInternal    738929
Census_ThresholdOptIn         564613
Census_IsWIMBootEnabled       563872
dtype: int64

### Checando relação Missing - Target

In [13]:
def create_nan(dataframe, column_name, value):
    try:
        dataframe[column_name].cat.add_categories(value, inplace=True)
    except:
        pass

    dataframe[column_name].fillna(value, inplace=True)

def check_nan_target(dataframe, column_name,target_name):
    ### return grouṕby sum
    dataframe = dataframe[[column_name, target_name]].copy()
    
    create_nan(dataframe, column_name,-1)

    return dataframe.groupby([column_name]).mean()

In [14]:
for column in check_columns.keys():
    print(check_nan_target(df, column, 'HasDetections'))

                           HasDetections
DefaultBrowsersIdentifier               
-1.0                            0.501575
 1.0                            0.500000
 9.0                            1.000000
 10.0                           0.440000
 11.0                           1.000000
...                                  ...
 3194.0                         1.000000
 3195.0                         0.550564
 3204.0                         1.000000
 3211.0                         0.500000
 3213.0                         1.000000

[737 rows x 1 columns]
                        HasDetections
OrganizationIdentifier               
-1.0                         0.496254
 1.0                         0.479167
 2.0                         0.364964
 3.0                         0.481481
 4.0                         0.410072
...                               ...
 48.0                        0.525982
 49.0                        0.458884
 50.0                        0.604469
 51.0                    

In [15]:
remove_columns = ['Census_IsWIMBootEnabled', 'Census_ThresholdOptIn', 'Census_IsFlightingInternal',
                           'Census_InternalBatteryType', 'DefaultBrowsersIdentifier']

df.drop(columns=remove_columns, inplace=True)

In [16]:
create_columns = ['SmartScreen', 'SMode', 'OrganizationIdentifier']

# setando missing como categoria
for column in create_columns:
    create_nan(df, column, -1)

In [17]:
df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,889744.00000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,...,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000,889744.000000
mean,0.00001,6.846086,0.017303,47862.886494,1.325295,1.020654,0.987865,108.060740,83025.476684,16.894997,...,397.261119,33013.334262,0.485751,0.006936,0.125647,0.038060,0.057080,0.274109,7.948012,0.499486
std,0.00318,1.023502,0.130397,14006.732586,0.521869,0.166729,0.109488,63.054879,48947.702438,12.818502,...,222.464246,21019.459893,0.499797,0.082992,0.331452,0.191342,0.231996,0.446065,4.491274,0.500000
min,0.00000,0.000000,0.000000,16.000000,1.000000,0.000000,0.000000,1.000000,7.000000,-1.000000,...,11.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.00000,7.000000,0.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,37761.000000,-1.000000,...,142.000000,13250.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.00000,7.000000,0.000000,53447.000000,1.000000,1.000000,1.000000,97.000000,84704.000000,18.000000,...,486.000000,33075.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000
75%,0.00000,7.000000,0.000000,53447.000000,2.000000,1.000000,1.000000,162.000000,130059.000000,27.000000,...,556.000000,52249.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,1.00000,35.000000,1.000000,70496.000000,6.000000,5.000000,1.000000,222.000000,167962.000000,52.000000,...,1084.000000,72091.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [23]:
df.to_csv("./data/train_un_pre.csv")